In [1]:
# Making imports convenient
import sys
import os
MODULE_FULL_PATH=os.getcwd().split('/notebooks')[0]
sys.path.insert(1, MODULE_FULL_PATH)

import torch
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding,RobertaForSequenceClassification,AdamW,get_scheduler,TrainingArguments,Trainer

from src.utils.myutils import *


device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

DATA_PATH = MODULE_FULL_PATH + '/data/CS/processed/WIKI-CS'
model_checkpoint = 'ufal/robeczech-base'
BATCH_SIZE = 32

## Data

In [2]:
train = load_dataset('csv',data_files=DATA_PATH+"/train.csv")['train']
test = load_dataset('csv',data_files=DATA_PATH+"/test.csv")['train']

Using custom data configuration default-56c437b671b1e68a
Reusing dataset csv (/home/horyctom/.cache/huggingface/datasets/csv/default-56c437b671b1e68a/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff)
Using custom data configuration default-8ec62ab79f9b4092
Reusing dataset csv (/home/horyctom/.cache/huggingface/datasets/csv/default-8ec62ab79f9b4092/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff)


## Training

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=False) #fast tokenizer is buggy in RoBERTa models
model = RobertaForSequenceClassification.from_pretrained(model_checkpoint)
model.to(device);

Some weights of the model checkpoint at ufal/robeczech-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at ufal/robeczech-base and are newly initialized: 

In [4]:
training_args = TrainingArguments(
    output_dir='../',
    num_train_epochs=3,
    per_device_train_batch_size=BATCH_SIZE,  
    logging_steps=25,
    disable_tqdm = False,
    save_total_limit=2,
    learning_rate=5e-5)

In [7]:
train_tokenized = preprocess_data(train,tokenizer)
test_tokenized = preprocess_data(test,tokenizer)

Loading cached processed dataset at /home/horyctom/.cache/huggingface/datasets/csv/default-56c437b671b1e68a/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff/cache-af7e92065c896c17.arrow
Loading cached processed dataset at /home/horyctom/.cache/huggingface/datasets/csv/default-8ec62ab79f9b4092/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff/cache-ebd98b4933fb3af7.arrow


In [6]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [8]:
trainer = Trainer(model,training_args,train_dataset=train_tokenized,data_collator=data_collator,tokenizer=tokenizer)
trainer.train()

***** Running training *****
  Num examples = 12368
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 1161


Step,Training Loss
25,0.696300
50,0.694300
75,0.691600
100,0.690100
125,0.689800


RuntimeError: The expanded size of the tensor (1366) must match the existing size (514) at non-singleton dimension 1.  Target sizes: [32, 1366].  Tensor sizes: [1, 514]